<a href="https://colab.research.google.com/github/drew-walkerr/Diss_Detecting_Provider_Bias/blob/main/doubt_markers_regex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import os
import re
import sys
import scipy
import Cython
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import matplotlib.pyplot as plt
import spacy
from spacy.lang.en import English
import numpy as np

from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/gdrive/')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [84]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding


In [85]:

%cd /content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/1_Data Prep
%ls

/content/gdrive/MyDrive/Diss_Detecting_Provider_Bias/Aim 1/Doubt Markers/1_Data Prep
1_doubt_makers_word_embeddings.py
doubt_markers_dataframe_annotate.csv
doubt_markers_lexicon_dev.ipynb
doubt_markers_regex.ipynb
doubt_word_list_round_1_dw_pruned.csv
doubt_word_list_round_2.csv
doubt_words_lexicon_stem_and_similar_round1.csv
expanded_misspellings.csv
lexicon_search_and_visualizations_subreddit.Rmd
trig-vectors-phrase.bin


BigQuery

In [86]:
#Replace 'project_id' with your BigQuery project ID

from google.cloud import bigquery
client = bigquery.Client(project='detecting-bias-mimic-iii')

icd_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.diagnoses_icd
                LIMIT 1000''')

icd = client.query(icd_query).to_dataframe()



ICD codes

In [87]:


#Filter for ICD codes for 282.60-282.69, referring to sickle cell types w/wo crisis
#2824 for thalassemia w + w/o crisis (282.41-282.42)
#SCD: 2826,2824
#Chronic Pain 3382
#Opioid dependencies: 3040,3047 (combo),
#HIV/AIDS ^042$

icds_of_interest = icd[icd['ICD9_CODE'].str.contains('2826|2824|3040|3047|3382|^042$', na=False)]
# Issues here-- will cause more leading numbers
print(icds_of_interest.head())

icds_of_interest.info()

#patients_unique = icds_of_interest['SUBJECT_ID'].drop_duplicates()


     ROW_ID  SUBJECT_ID  HADM_ID  SEQ_NUM ICD9_CODE
784  176454       15872   139593        3     73382
889  196755       17664   123887        7     33829
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 784 to 889
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ROW_ID      2 non-null      Int64 
 1   SUBJECT_ID  2 non-null      Int64 
 2   HADM_ID     2 non-null      Int64 
 3   SEQ_NUM     2 non-null      Int64 
 4   ICD9_CODE   2 non-null      object
dtypes: Int64(4), object(1)
memory usage: 104.0+ bytes


Notes and patients load

In [88]:
note_query = ('''SELECT *
                FROM physionet-data.mimiciii_notes.noteevents''')
                # LIMIT 500000

NOTES = client.query(note_query).to_dataframe()

peek_notes = NOTES.head()
print(peek_notes)

## Patients

patient_query = ('''SELECT *
                FROM physionet-data.mimiciii_clinical.patients''')

PATIENTS = client.query(patient_query).to_dataframe()

peek_patients = PATIENTS.head()
print(peek_patients)

biased_notes_patients_corpus = NOTES.merge(PATIENTS, on = 'SUBJECT_ID')

biased_notes_patients_corpus.info()
# Find unique chart types and decide to limit them
    # Remove Radiology, ECG, Respiratory, Echo notes
biased_notes_patients_corpus_filtered = biased_notes_patients_corpus[biased_notes_patients_corpus["CATEGORY"].str.contains("Radiology|ECG|Respiratory|Echo")==False]
# biased_notes_patients_corpus_filtered.to_csv("biased_notes_patients_corpus_filtered.csv")



    ROW_ID  SUBJECT_ID  HADM_ID   CHARTDATE           CHARTTIME  \
0  1557640       22599   164788  2184-03-10 2184-03-10 04:27:00   
1  1892226       15204   168396  2128-01-25 2128-01-25 05:58:00   
2  1974608       21464   109525  2189-05-13 2189-05-13 10:56:00   
3  1993393       23286   122536  2141-07-14 2141-07-14 11:34:00   
4  1848162       11994   142976  2104-11-04 2104-11-04 06:42:00   

            STORETIME       CATEGORY DESCRIPTION   CGID  ISERROR  \
0 2184-03-10 04:38:00  Nursing/other      Report  14962     <NA>   
1 2128-01-25 06:10:00  Nursing/other      Report  15484     <NA>   
2 2189-05-13 10:58:00  Nursing/other      Report  15444     <NA>   
3 2141-07-14 11:40:00  Nursing/other      Report  14943     <NA>   
4 2104-11-04 06:51:00  Nursing/other      Report  14470     <NA>   

                                                TEXT  
0  csru nursing update\nadditional lopressor iv b...  
1  NPN\n\n\n#2Resp O- Remains in NC 600cc's 70-10...  
2  Neo Attend PE note\n

Explode by sentence, regex quote match

In [89]:
full_dataframe = biased_notes_patients_corpus_filtered
# Tokenize by sentence
nlp = English()  # just the language with no model

nlp.add_pipe('sentencizer')
full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])
full_dataframe = full_dataframe.explode("Sentence", ignore_index=True)
full_dataframe.rename(columns={"Unnamed: 0": "ROW_ID_new"}, inplace=True)
full_dataframe.index.name = "Sentence ID"

full_dataframe['Sentence'].replace(r'\s+|\\n', ' ', regex=True, inplace=True)



<ipython-input-89-1e5fd6deed48>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_dataframe["Sentence"] = full_dataframe["TEXT"].apply(lambda x: [sent.text for sent in nlp(x).sents])


# Regex matching all stemwords + any semantically similar word identified

In [90]:


doubt_word_list = pd.read_csv("doubt_word_list_round_2.csv")


# Create regex which is each stem word + all similar_words identified in the word embeddings step, which will be used to search the charts
def group_and_create_regex(dataframe):
    grouped_data = dataframe.groupby('stem_word')['similar_word'].apply(list).reset_index()
    grouped_data['regex'] = grouped_data.apply(lambda row: '|'.join([row['stem_word']] + row['similar_word']), axis=1)
    return grouped_data

grouped_data = group_and_create_regex(doubt_word_list)
print(grouped_data)

# Create function that takes regex from each row and searches the charts
def identify_matching_strings(full_dataframe, doubt_word_list):
    matching_dataframes = []

    for index, row in doubt_word_list.iterrows():
        regex = row['regex']
        stem_word = row['stem_word']
        matches = full_dataframe.loc[full_dataframe['Sentence'].str.contains(regex, flags=re.IGNORECASE)].copy()
        matches['regex'] = regex
        matches['stem_word'] = stem_word
        matching_dataframes.append(matches)

    if matching_dataframes:
        return pd.concat(matching_dataframes, axis=0)
    else:
        return pd.DataFrame()


result = identify_matching_strings(full_dataframe, grouped_data)
print(result)
result




     stem_word                                       similar_word  \
0      adamant         [adamantly, adamant/belligerant, addamant]   
1    allegedly  [alleged, supposedly, undisguisedly, purported...   
2      claimed  [claiming, proclaimed, claimes, claimedthat, a...   
3  disbelieves  [disbelieve, misbelieves, disbelieved, disbeli...   
4      dubious  [doubious, 'dubious, non-dubious, dubiously, d...   
5      insists  [insisting, reinsists, insist, insisted, .insi...   

                                               regex  
0     adamant|adamantly|adamant/belligerant|addamant  
1  allegedly|alleged|supposedly|undisguisedly|pur...  
2  claimed|claiming|proclaimed|claimes|claimedtha...  
3  disbelieves|disbelieve|misbelieves|disbelieved...  
4  dubious|doubious|'dubious|non-dubious|dubiousl...  
5  insists|insisting|reinsists|insist|insisted|.i...  
             ROW_ID_x  SUBJECT_ID  HADM_ID   CHARTDATE           CHARTTIME  \
Sentence ID                                          

,ROW_ID_x,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,...,ROW_ID_y,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG,Sentence,regex,stem_word
Sentence ID,,,,,,,,,,,,,,,,,,,,,
71975,1345578,6334,115026,2168-02-02,2168-02-02 16:03:00,2168-02-02 16:24:00,Nursing/other,Report,19689,<NA>,...,5998,M,2097-09-08,2168-02-09,2168-02-09,2168-02-09,1,"PT worked c pt for nearly two hrs, unfortunat...",adamant|adamantly|adamant/belligerant|addamant,adamant
72700,1345580,6334,115026,2168-02-03,2168-02-03 16:00:00,2168-02-03 16:31:00,Nursing/other,Report,19689,<NA>,...,5998,M,2097-09-08,2168-02-09,2168-02-09,2168-02-09,1,"Pt incont of urine on two occasions today, pt...",adamant|adamantly|adamant/belligerant|addamant,adamant
144917,1523903,19911,180900,2130-11-13,2130-11-13 05:12:00,2130-11-13 05:32:00,Nursing/other,Report,20088,<NA>,...,18797,F,2066-03-22,2132-11-01,NaT,2132-11-01,1,SOCIAL: SPOKE WITH DAUGHTER [**Name (NI) **] ...,adamant|adamantly|adamant/belligerant|addamant,adamant
183114,389565,27800,168730,2160-11-11,2160-11-11 15:41:00,2160-11-11 15:41:17,Social Work,Social Work Progress Note,20322,<NA>,...,26217,M,2096-05-09,NaT,NaT,NaT,0,He shares intubation is fine for him as long ...,adamant|adamantly|adamant/belligerant|addamant,adamant
190936,11669,11721,138312,2193-11-29,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,11089,F,2120-02-01,2200-11-28,NaT,2200-11-28,1,She stated that she adamantly would refuse an...,adamant|adamantly|adamant/belligerant|addamant,adamant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30552850,48358,86477,163070,2123-03-21,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,42983,M,2060-04-22,NaT,NaT,NaT,0,"He had no cat exposures, but had planted a gar...",insists|insisting|reinsists|insist|insisted|.i...,insists
30560618,52102,11385,151452,2185-03-16,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,10764,F,2102-06-20,2185-03-16,2185-03-16,2185-03-16,1,She insists that she had no presyncopal events.,insists|insisting|reinsists|insist|insisted|.i...,insists
30569707,48113,95239,157940,2137-01-23,NaT,NaT,Discharge summary,Report,<NA>,<NA>,...,45272,F,2081-05-05,NaT,NaT,NaT,0,Tob: Less than one pack per week Family Histo...,insists|insisting|reinsists|insist|insisted|.i...,insists


In [91]:

quoted_dataframe = result

quoted_dataframe["doubt_testimony"] = ""
quoted_dataframe["annotator_comments"] = ""
# 1459 matching sentences from total of 9,118,501 sentences from 366,002 charts
quoted_dataframe2 = quoted_dataframe.drop_duplicates(subset=['Sentence'])
quoted_dataframe2.to_csv("doubt_markers_dataframe_annotate.csv")



Explore the Data


In [92]:
#from google.colab import runtime
#runtime.unassign()